In [1]:
%matplotlib inline

# Load the modules
import pickle
import math

import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt

# Reload the data
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  train_features = pickle_data['train_dataset']
  train_labels = pickle_data['train_labels']
  valid_features = pickle_data['valid_dataset']
  valid_labels = pickle_data['valid_labels']
  test_features = pickle_data['test_dataset']
  test_labels = pickle_data['test_labels']
  del pickle_data  # Free up memory


print('Data and modules loaded.')

Data and modules loaded.


In [2]:

# Data has been normalized
total_samples = len(train_features)

# Parameters
learning_rate = 0.01
num_output = 10
epochs = 40
batch_size = 256
steps_per_epoch = int(np.ceil(total_samples / batch_size))

In [3]:
# Cleaning Data
#valid_labels, test_labels = np.array(valid_labels, np.int64), np.array(test_labels, np.int64)
train_data = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
val_features, test_features = np.array(valid_features, np.float32), np.array(test_features, np.float32)

# Data has been normalized

In [4]:
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [5]:
features = train_features.shape[1]
print(total_samples)

142500


In [6]:
#W = tf.Variable(tf.random.normal([features, num_output], name="weights1"))
#B = tf.Variable(tf.zeros([num_output], name="bias1"))
mu = 0
sigma = 0.001

W = {
    "w1": tf.Variable(tf.random.normal([features, 128], mean=mu, stddev=sigma), name="weight1"),
    "w2": tf.Variable(tf.random.normal([128, 32], mean=mu, stddev=sigma), name="weight2"),
    #"w3": tf.Variable(tf.random.normal([128, 32], mean=mu, stddev=sigma), name="weight3"),
    "out": tf.Variable(tf.random.normal([32, num_output], mean=mu, stddev=sigma), name="weight_out")
}

B = {
    "b1": tf.Variable(tf.zeros([128], name="bias1")),
    "b2": tf.Variable(tf.zeros([32], name="bias2")),
    #"b3": tf.Variable(tf.zeros([32], name="bias3")),
    "out": tf.Variable(tf.zeros([num_output], name="bias_out"))
}

optimizer = tf.optimizers.Adam(learning_rate)

# Model
class MyModel(tf.Module):
    def __call__(self, X):
      X = tf.add(tf.matmul(X, W["w1"]), B["b1"])
      X = tf.nn.relu(X)
      X = tf.add(tf.matmul(X, W["w2"]), B["b2"])
      X = tf.nn.relu(X)
      #X = tf.add(tf.matmul(X, W["w3"]), B["b3"])
      #X = tf.nn.relu(X)
      return tf.nn.softmax(tf.add(tf.matmul(X, W["out"]), B["out"]))

def cross_entropy(y_pred, y_true):
    # It has been one-hot encoded before storing as pickle
    #y_true = tf.one_hot(y_true, depth=num_output)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

def accuracy(y_pred, y_true):
    if len(y_true.shape) > 1 and y_true.shape[1] > 1:
        y_true = tf.argmax(y_true, axis=1)
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def run_optimizer(X, Y):
    with tf.GradientTape() as g:
        logit = model(X)
        loss = cross_entropy(logit, Y)

    trainable_variables = list(W.values()) + list(B.values())

    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return None

def batch_data(X, Y, batch_size):
    output_data = []
    sample_size = len(X)
    for step in range(0, sample_size, batch_size):
        start = batch_size * step
        end = batch_size + start
        batch_X = X[start:end]
        batch_Y = Y[start:end]
        yield batch_X, batch_Y

model=MyModel()

checkpoint = tf.train.Checkpoint(model = model)
checkpoint.save("./checkpoints/mymodel")
manager = tf.train.CheckpointManager(checkpoint, "./checkpoints", max_to_keep=1)

In [7]:
# Training
for epoch in range(1, epochs + 1):
    for step, (batch_X, batch_Y) in enumerate(train_data.take(steps_per_epoch), 1):
        run_optimizer(batch_X, batch_Y)

    val_pred = model(valid_features)
    val_loss = cross_entropy(val_pred, valid_labels)
    val_acc = accuracy(val_pred, valid_labels)
    train_pred = model(batch_X)
    train_loss = cross_entropy(train_pred, batch_Y)
    train_acc = accuracy(train_pred, batch_Y)
    manager.save()
    new_learning_rate = learning_rate * (0.95 ** epoch)
    optimizer.learning_rate.assign(new_learning_rate)


    print(f"Epoch: {epoch}, train_loss {train_loss}, train_accuracy {train_acc}, val_loss {val_loss}, val_accuracy {val_acc}")

Epoch: 1, train_loss 0.4027448892593384, train_accuracy 0.890625, val_loss 0.5390413999557495, val_accuracy 0.8420000076293945
Epoch: 2, train_loss 0.5751396417617798, train_accuracy 0.8203125, val_loss 0.4794931709766388, val_accuracy 0.8555999994277954
Epoch: 3, train_loss 0.42961764335632324, train_accuracy 0.84375, val_loss 0.4768078327178955, val_accuracy 0.8589333295822144
Epoch: 4, train_loss 0.5140156149864197, train_accuracy 0.83984375, val_loss 0.4674294888973236, val_accuracy 0.8622666597366333
Epoch: 5, train_loss 0.33536675572395325, train_accuracy 0.8984375, val_loss 0.4455438554286957, val_accuracy 0.8665333390235901
Epoch: 6, train_loss 0.3073121905326843, train_accuracy 0.9140625, val_loss 0.42466306686401367, val_accuracy 0.8704000115394592
Epoch: 7, train_loss 0.4008389115333557, train_accuracy 0.8828125, val_loss 0.424173504114151, val_accuracy 0.8733333349227905
Epoch: 8, train_loss 0.31074684858322144, train_accuracy 0.90625, val_loss 0.42650213837623596, val_accu

In [8]:
test_pred = model(test_features)
test_loss = cross_entropy(test_pred, test_labels)
test_acc = accuracy(test_pred, test_labels)
print(f"Test_loss {test_loss}, Test_accuracy {test_acc}")

Test_loss 0.2845991551876068, Test_accuracy 0.9287999868392944
